In [70]:
import os
import matplotlib.pyplot as plt
import numpy as np
from src.utils.dicom import *
from src.utils.crop import *
from glob import glob

data_dir = '/mnt/d/datasets/CBIS/data'
mask_sample_path = os.path.join(data_dir, 'CBIS-DDSM/Mass-Training_P_01221_LEFT_CC_1/07-21-2016-DDSM-NA-02934/1.000000-ROI mask images-77280')
img_sample_path = os.path.join(data_dir, 'CBIS-DDSM/Mass-Training_P_01221_LEFT_CC/07-20-2016-DDSM-NA-68911/1.000000-full mammogram images-82675')

# 1. Load image
dcm_img = glob(img_sample_path + '/*.dcm')
img_1 = load_dicom_image(dcm_img[0])

# 2. Get list of files in roi paths
dcm_mask_files = glob(mask_sample_path + '/*.dcm')

# 3. Load the mask file (the one with the same shape as the original image)
mask = load_dicom_mask(dcm_mask_files, img_1.shape)

# 4. Extract patch
patch = extract_patch(img_1, mask)

# 5. Random patches several times
crops_size = min(patch.shape) - 10
patches = [random_crop(patch, size=(crops_size, crops_size)) for i in range(3)]
